In [1]:
# import modules
import uproot, sys, time, math, pickle, os
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import awkward as ak
from tqdm import tqdm
import seaborn as sns
from sklearn.metrics import roc_curve, auc
from sklearn.model_selection import train_test_split
from matplotlib.ticker import FormatStrFormatter
import matplotlib.ticker as ticker
from scipy.special import betainc
from scipy.stats import norm

# import config functions
sys.path.append("/home/jlai/jet-faking/config")
from jet_faking_plot_config import getWeight, zbi, sample_dict, getVarDict
from plot_var import variables, variables_data, ntuple_names, ntuple_names_BDT
from n_1_iteration_functions import get_best_cut, calculate_significance, apply_cut_to_fb, apply_all_cuts, compute_total_significance, n_minus_1_optimizer
# from cut_config import cut_config

# Set up plot defaults
import matplotlib as mpl
mpl.rcParams['figure.figsize'] = 14.0,10.0  # Roughly 11 cm wde by 8 cm high  
mpl.rcParams['font.size'] = 20.0 # Use 14 point font
sns.set(style="whitegrid")

font_size = {
    "xlabel": 17,
    "ylabel": 17,
    "xticks": 15,
    "yticks": 15,
    "legend": 14
}

plt.rcParams.update({
    "axes.labelsize": font_size["xlabel"],  # X and Y axis labels
    "xtick.labelsize": font_size["xticks"],  # X ticks
    "ytick.labelsize": font_size["yticks"],  # Y ticks
    "legend.fontsize": font_size["legend"]  # Legend
})

tot = []
data = pd.DataFrame()
ntuple_names = ['ggHyyd','Zjets','Zgamma','Wgamma','Wjets','gammajet_direct', 'data23']

def test(fb):
    # checking if there are any none values
    mask = ak.is_none(fb['met_tst_et'])
    n_none = ak.sum(mask)
    print("Number of none values: ", n_none)
    # if n_none > 0:
    #     fb = fb[~mask]
    # print("Events after removing none values: ", len(fb), ak.sum(ak.is_none(fb['met_tst_et'])))

def print_cut(ntuple_name, fb, label):
    print(f"Unweighted Events {label}: ", len(fb))
    if ntuple_name == 'data23':
        print(f"Weighted Events {label}: ", sum(getWeight(fb, ntuple_name, jet_faking=True)))
    else: 
        print(f"Weighted Events {label}: ", sum(getWeight(fb, ntuple_name)))

for i in range(len(ntuple_names)):
    start_time = time.time()
    ntuple_name = ntuple_names[i]
    if ntuple_name == 'data23': # data
        path = f"/data/fpiazza/ggHyyd/Ntuples/MC23d/withVertexBDT/data23_y_BDT_score.root" 
        print('processing file: ', path)
        f = uproot.open(path)['nominal']
        fb = f.arrays(variables_data, library="ak")
        fb['VertexBDTScore'] = fb['BDTScore'] # renaming BDTScore to ensure this is recognized as Vertex BDT Score
        
        fb = fb[ak.num(fb['ph_eta']) > 0]     # for abs(ak.firsts(fb['ph_eta'])) to have value to the reweighting
                
        mask1 = (ak.firsts(fb['ph_topoetcone40'])-2450.)/ak.firsts(fb['ph_pt']) > 0.1   # jet_faking_photon cut
        fb = fb[mask1]
        fb = fb[fb['n_ph_baseline'] == 1]

    else: # MC
        path = f"/data/tmathew/ntups/mc23d/{ntuple_name}_y.root" 
        path_BDT = f"/data/fpiazza/ggHyyd/Ntuples/MC23d/withVertexBDT/mc23d_{ntuple_name}_y_BDT_score.root" 
        print('processing file: ', path)
        f = uproot.open(path)['nominal']
        fb = f.arrays(variables, library="ak")

        # add BDT score to fb
        f_BDT = uproot.open(path_BDT)['nominal']
        fb_BDT = f_BDT.arrays(["event", "BDTScore"], library="ak")
        tmp = fb["event"] == fb_BDT["event"]
        if np.all(tmp) == True:
            fb["VertexBDTScore"] = fb_BDT["BDTScore"]
        else: 
            print("Something is wrong, need arranging")

        fb = fb[ak.num(fb['ph_eta']) > 0]     # for abs(ak.firsts(fb['ph_eta'])) to have value to the reweighting
        fb = fb[fb['n_ph'] == 1]
        
        # Zjets and Wjets (rule out everything except for e->gamma)
        if ntuple_name == 'Zjets' or ntuple_name == 'Wjets':
            mask = ak.firsts(fb['ph_truth_type']) == 2
            fb = fb[mask]
        
        # goodPV on signal only
        if ntuple_name == 'ggHyyd':
            fb = fb[ak.num(fb['pv_z']) > 0]
            good_pv_tmp = (np.abs(ak.firsts(fb['pv_truth_z']) - ak.firsts(fb['pv_z'])) <= 0.5)
            fb = fb[good_pv_tmp]

    print_cut(ntuple_name, fb, 'before cut')

    fb = fb[fb['n_mu_baseline'] == 0]
    fb = fb[fb['n_el_baseline'] == 0]
    fb = fb[fb['n_tau_baseline'] == 0]
    fb = fb[fb['trigger_HLT_g50_tight_xe40_cell_xe70_pfopufit_80mTAC_L1eEM26M']==1]
    fb = fb[ak.num(fb['ph_pt']) > 0] # prevent none values in Tbranch
    fb = fb[ak.firsts(fb['ph_pt']) >= 50000] # ph_pt cut (basic cut)
    fb = fb[fb['met_tst_et'] >= 100000] # MET cut (basic cut)
    fb = fb[fb['n_jet_central'] <= 3] # n_jet_central cut (basic cut)

    mt_tmp = np.sqrt(2 * fb['met_tst_et'] * ak.firsts(fb['ph_pt']) * 
                            (1 - np.cos(fb['met_tst_phi'] - ak.firsts(fb['ph_phi'])))) / 1000
    mask1 = mt_tmp > 100
    mask2 = mt_tmp < 140 
    fb = fb[mask1 * mask2]

    fb = fb[fb['VertexBDTScore'] > 0.1]

    '''
    # Selection cut
    metsig_tmp = fb['met_tst_sig'] 
    mask1 = metsig_tmp > 7
    mask2 = metsig_tmp < 20
    fb = fb[mask1 * mask2]
    
    ph_eta_tmp = np.abs(ak.firsts(fb['ph_eta']))
    fb = fb[ph_eta_tmp < 1.74]

    dphi_met_phterm_tmp = np.arccos(np.cos(fb['met_tst_phi'] - fb['met_phterm_phi'])) # added cut 3
    fb = fb[dphi_met_phterm_tmp > 1.23]

    dmet_tmp = fb['met_tst_noJVT_et'] - fb['met_tst_et']
    mask1 = dmet_tmp > -21000
    mask2 = dmet_tmp < 41600
    fb = fb[mask1 * mask2]

    dphi_jj_tmp = fb['dphi_central_jj']
    dphi_jj_tmp = ak.where(dphi_jj_tmp == -10, np.nan, dphi_jj_tmp)
    dphi_jj_tmp = np.arccos(np.cos(dphi_jj_tmp))
    dphi_jj_tmp = ak.where(np.isnan(dphi_jj_tmp), -999, dphi_jj_tmp)
    fb = fb[dphi_jj_tmp < 2.37]

    dphi_met_jetterm_tmp = np.where(fb['met_jetterm_et'] != 0,   # added cut 5
                        np.arccos(np.cos(fb['met_tst_phi'] - fb['met_jetterm_phi'])),
                        -999)
    fb = fb[dphi_met_jetterm_tmp < 0.73]

    # print_cut(ntuple_name, fb, 'after basic + selection cut')
    '''
    print_cut(ntuple_name, fb, 'after basic')

    test(fb) # check for none value

    print(f"Reading Time for {ntuple_name}: {(time.time()-start_time)} seconds\n")


    tot.append(fb)

    fb = 0
    fb_BDT = 0
    tmp = 0


processing file:  /data/tmathew/ntups/mc23d/ggHyyd_y.root
Unweighted Events before cut:  86910
Weighted Events before cut:  8732.987955115426
Unweighted Events after basic:  2646
Weighted Events after basic:  268.2449529933159
Number of none values:  0
Reading Time for ggHyyd: 1.7772290706634521 seconds

processing file:  /data/tmathew/ntups/mc23d/Zjets_y.root
Unweighted Events before cut:  3242488
Weighted Events before cut:  676616.903247458
Unweighted Events after basic:  383
Weighted Events after basic:  16.82812304884456
Number of none values:  0
Reading Time for Zjets: 90.09477019309998 seconds

processing file:  /data/tmathew/ntups/mc23d/Zgamma_y.root
Unweighted Events before cut:  3423357
Weighted Events before cut:  249851.55031619867
Unweighted Events after basic:  19491
Weighted Events after basic:  1002.7013259970018
Number of none values:  0
Reading Time for Zgamma: 22.769730806350708 seconds

processing file:  /data/tmathew/ntups/mc23d/Wgamma_y.root
Unweighted Events befo

In [2]:
signal_name = 'ggHyyd'  # Define signal dataset
cut_name = 'basic'  

def getCutDict():
    cut_dict = {}
    # Selection 1: same variables as in the internal note
    cut_dict['dmet'] = {
        'lowercut': np.arange(-30000, 10000 + 100, 100), # dmet > cut
        'uppercut': np.arange(10000, 100000 + 100, 100), # -10000 < dmet < cut
    }
    cut_dict['metsig'] = {
        'lowercut': np.arange(0, 10 + 1, 1), # metsig > cut
        'uppercut': np.arange(10, 30 + 1, 1), # metsig < cut 
    }
    cut_dict['dphi_met_phterm'] = {
        'lowercut': np.arange(1, 2 + 0.01, 0.01), # dphi_met_phterm > cut
    }
    cut_dict['dphi_met_jetterm'] = {
        'uppercut': np.arange(0.5, 1, 0.01), # dphi_met_jetterm < cut
    }
    cut_dict['ph_eta'] = {
        'uppercut': np.arange(1, 2.5 + 0.01, 0.01), # ph_eta < cut
    }
    cut_dict['dphi_jj'] = {
        'uppercut': np.arange(1, 3.14 + 0.01, 0.01) # dphi_jj < cut
    }

    # Selection 2
    cut_dict['balance'] = {
        'lowercut': np.arange(0.2, 1.5 + 0.01, 0.01), # balance > cut
        'uppercut': np.arange(0.5, 3, 0.01) # balance < cut
    }
    cut_dict['jetterm'] = {
        'lowercut': np.arange(0, 150000+1000, 1000) # jetterm > cut
    }
    cut_dict['ph_pt'] = {
        'lowercut': np.arange(50000, 100000 + 1000, 1000),  # ph_pt > cut
        'uppercut': np.arange(100000, 300000 + 1000, 1000),  # ph_pt > cut
    }
    cut_dict['dphi_phterm_jetterm'] = {
        'lowercut': np.arange(1, 2.5 + 0.05, 0.05), # dphi_phterm_jetterm > cut
        'uppercut': np.arange(2, 4 + 0.1, 0.1) # dphi_phterm_jetterm < cut
    }
    cut_dict['metsigres'] = {
        'uppercut': np.arange(12000, 60000, 1000)
    }
    cut_dict['met_noJVT'] = {
        'lowercut': np.arange(50000, 120000, 1000),
    }
    
    return cut_dict
cut_config = getCutDict()

In [3]:
%%time
signal_name='ggHyyd'
initial_cut = []
tot2 = tot

# < -- Initial Cut on all variables (maximize the significance * acceptance) -- > 
for cut_var, cut_types in cut_config.items():
    for cut_type, cut_values in cut_types.items():
        sig_simple_list, sigacc_simple_list, acceptance_values = calculate_significance(
            cut_var, cut_type, cut_values, tot2, ntuple_names, signal_name, getVarDict, getWeight
        )

        best_cut, best_sig, idx = get_best_cut(cut_values, sigacc_simple_list) 
        
        if idx == 0 or idx == len(sigacc_simple_list) - 1: # I chose to use index to indicate not to make unnecessary cut (for initial cut)
            print(cut_var, idx, len(sigacc_simple_list))
            continue
            
        result = {
            "cut_var": cut_var,
            "cut_type": cut_type,
            "best_cut": best_cut,
            "best_sig_x_acc": best_sig,
            "significance": sig_simple_list[idx],
            "acceptance": acceptance_values[idx]
        }

        print(result)
        initial_cut.append(dict(list(result.items())[:3]))

{'cut_var': 'dmet', 'cut_type': 'lowercut', 'best_cut': -19400, 'best_sig_x_acc': 1.8218156666685783, 'significance': 1.8370676880976597, 'acceptance': 99.1697626860513}
{'cut_var': 'dmet', 'cut_type': 'uppercut', 'best_cut': 41800, 'best_sig_x_acc': 1.7533151838946883, 'significance': 1.7537552502018237, 'acceptance': 99.97490719944618}
{'cut_var': 'metsig', 'cut_type': 'lowercut', 'best_cut': 6, 'best_sig_x_acc': 2.3725553020181507, 'significance': 2.5828703809512277, 'acceptance': 91.85731190832652}
{'cut_var': 'metsig', 'cut_type': 'uppercut', 'best_cut': 20, 'best_sig_x_acc': 1.7512994164399953, 'significance': 1.7512983444608226, 'acceptance': 100.00006121053995}
{'cut_var': 'dphi_met_phterm', 'cut_type': 'lowercut', 'best_cut': 1.03, 'best_sig_x_acc': 1.7935629035994418, 'significance': 1.8177016513714308, 'acceptance': 98.67201816349913}
{'cut_var': 'dphi_met_jetterm', 'cut_type': 'uppercut', 'best_cut': 0.8800000000000003, 'best_sig_x_acc': 1.7517103097746178, 'significance': 

In [4]:
tot2_initial_cut = apply_all_cuts(tot2, ntuple_names, initial_cut, getVarDict)
final_significance = compute_total_significance(tot2_initial_cut, ntuple_names, signal_name, getVarDict, getWeight)
print('after initial cutting, signficance: ', final_significance)

after initial cutting, signficance:  2.733286501616026


In [5]:
%%time

def n_minus_1_optimizer(
    initial_cut,
    cut_config,
    tot2,
    ntuple_names,
    signal_name,
    getVarDict,
    getWeight,
    final_significance,
    max_iter=10,
    tolerance=1e-4,
    allow_drop=True,
    drop_tolerance=1e-6
):
    """
    allow_drop: if True, remove a cut when N-1 significance beats the best-with-cut significance.
    drop_tolerance: minimal margin by which N-1 must exceed best-with-cut to drop the cut.
    """
    best_cuts = [dict(c) for c in initial_cut]  # copy
    iteration = 0

    while iteration < max_iter:
        converged = True
        to_remove = []

        print(f"\n--- Iteration {iteration + 1} ---")

        for i, cut in enumerate(best_cuts):
            # Apply all OTHER cuts (N-1)
            n_minus_1_cuts = best_cuts[:i] + best_cuts[i+1:]
            tot2_cut = apply_all_cuts(tot2, ntuple_names, n_minus_1_cuts, getVarDict)

            # Significance WITHOUT this cut
            sig_without = compute_total_significance(
                tot2_cut, ntuple_names, signal_name, getVarDict, getWeight
            )

            # Re-scan this variable ON TOP of N-1
            cut_var  = cut["cut_var"]
            cut_type = cut["cut_type"]
            scan_vals = cut_config[cut_var][cut_type]

            sig_simple_list, sigacc_simple_list, _ = calculate_significance(
                cut_var, cut_type, scan_vals, tot2_cut, ntuple_names,
                signal_name, getVarDict, getWeight
            )
            best_cut_val, best_sig, best_idx = get_best_cut(scan_vals, sig_simple_list)

            # Decide to drop or to keep/update
            if allow_drop and (sig_without >= best_sig + drop_tolerance):
                print(f"Dropping {cut_var} ({cut_type}): "
                      f"N-1 {sig_without:.3f} >= best-with-cut {best_sig:.3f}")
                to_remove.append(i)
                final_significance = sig_without
                converged = False
                continue  # move to next cut

            # Keep: update threshold if it moved
            if abs(best_cut_val - cut["best_cut"]) > tolerance:
                print(f"Updating {cut_var} ({cut_type}): "
                      f"{cut['best_cut']} → {best_cut_val}  "
                      f"(N-1 {sig_without:.3f} → with-cut {best_sig:.3f})")
                best_cuts[i]["best_cut"] = float(best_cut_val)
                final_significance = best_sig
                converged = False

        # Remove cuts marked for deletion (highest index first)
        if to_remove:
            for j in sorted(to_remove, reverse=True):
                del best_cuts[j]

        iteration += 1
        if converged:
            break

    # Recompute final significance with the surviving cuts
    tot2_final = apply_all_cuts(tot2, ntuple_names, best_cuts, getVarDict)
    final_significance = compute_total_significance(
        tot2_final, ntuple_names, signal_name, getVarDict, getWeight
    )

    print('optimized cuts, end of iteration')
    return best_cuts, final_significance

# < -- n-1 iterations until no further improvement (max significance) -- >
optimized_cuts, final_significance = n_minus_1_optimizer(
    initial_cut, cut_config, tot2, ntuple_names, signal_name, getVarDict, getWeight, final_significance
)
print('after optimized cutting, signficance: ', final_significance)



--- Iteration 1 ---
Updating dmet (uppercut): 41800 → 41700  (N-1 2.732 → with-cut 2.733)
Updating metsig (lowercut): 6 → 7  (N-1 2.559 → with-cut 2.804)
Updating metsig (uppercut): 20 → 17  (N-1 2.804 → with-cut 2.805)
Updating dphi_met_phterm (lowercut): 1.03 → 1.2000000000000002  (N-1 2.815 → with-cut 2.852)
Updating dphi_met_jetterm (uppercut): 0.8800000000000003 → 0.7300000000000002  (N-1 2.851 → with-cut 2.854)
Updating ph_eta (uppercut): 2.3500000000000014 → 1.7400000000000007  (N-1 2.838 → with-cut 3.055)
Dropping dphi_jj (uppercut): N-1 3.057 >= best-with-cut 2.401
Updating balance (lowercut): 0.6300000000000003 → 0.9100000000000006  (N-1 2.976 → with-cut 3.162)
Dropping balance (uppercut): N-1 3.227 >= best-with-cut 3.218
Updating jetterm (lowercut): 83000 → 81000  (N-1 3.164 → with-cut 3.164)
Dropping ph_pt (uppercut): N-1 3.177 >= best-with-cut 3.164
Updating dphi_phterm_jetterm (lowercut): 1.4500000000000004 → 1.6000000000000005  (N-1 3.164 → with-cut 3.166)
Updating dphi

In [7]:
print( ' < -- Final Optimized Cuts -- > ')
# print(optimized_cuts)

for cut in optimized_cuts:
    var = cut['cut_var']
    val = cut['best_cut']
    if cut['cut_type'] == 'uppercut':
        print(f"{var} < {val}")
    elif cut['cut_type'] == 'lowercut':
        print(f"{var} > {val}")
        
print('after optimized cutting, signficance: ', final_significance)


 < -- Final Optimized Cuts -- > 
dmet > -19400
dmet < 41600.0
metsig > 7.0
metsig < 20.0
dphi_met_phterm > 1.2300000000000002
dphi_met_jetterm < 0.7500000000000002
ph_eta < 1.7400000000000007
balance > 0.9100000000000006
jetterm > 92000.0
dphi_phterm_jetterm > 1.6000000000000005
dphi_phterm_jetterm < 3.100000000000001
metsigres < 36000.0
met_noJVT > 90000.0
after optimized cutting, signficance:  3.303890341832656


In [27]:
tot2_optimized_cuts = apply_all_cuts(tot2, ntuple_names, optimized_cuts, getVarDict)

print('< -- Sum of weight each process -- >')

for i in range(len(tot2_optimized_cuts)):
    print(ntuple_names[i], sum(getWeight(tot2_optimized_cuts[i], ntuple_names[i])))

< -- Sum of weight each process -- >
ggHyyd 167.68508374299802
Zjets 1.249980680862791
Zgamma 419.8129254336955
Wgamma 798.4444671642431
Wjets 710.7393202873664
gammajet_direct 6.537942385999486
data23 666.2637042786454
